Set Up

In [20]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression

In [21]:
data = pd.read_csv('new_dataset.csv')  # Update with your CSV file path
products = data.to_dict(orient='records')
data.head(5)

,id,product_name,brand,subcategory,rate,reviewed,recom,price,description,how_to_use,ingredients,suitable_for,url_new
0,1,01 Radiance Skin Exfoliate Serum/Peeling Serum,Lysca,Serum & Essence,5.0,6 users,100%,37500,"Lysca Radiance Skin AHA 3%, BHA 1%, PHA 2% + W...",Gunakan setelah membasuh wajah. teteskan dan r...,"Aqua, Hamamelis Virginiana Extract, Glycerin, ...",Semua jenis kulit,https://storage.googleapis.com/skinnie-bucket/...
1,10,10% Cica + Watermelon Extract Face Serum,Dear Me Beauty,Serum & Essence,4.5,156 users,92%,129000,Serum yang sangat powerful untuk perawatan kul...,-,-,Semua jenis kulit,https://storage.googleapis.com/skinnie-bucket/...
2,100,Ace Genius Hydrating & Clarifying Toner,Aceskin,Toner,5.0,6 users,84%,129000,Aceskin Ace Genius Hydrating & Clarifying Tone...,"Setelah membersihkan wajah, semprotkan langsun...","Galactomyces Ferment Filtrate, PHA, Zinc PCA",Semua jenis kulit,https://storage.googleapis.com/skinnie-bucket/...
3,1000,Brighter Skin Barrier Pudding,Studio Tropik,Cream,4.5,4 users,100%,-,Studio Tropik Brighter Skin Barrier Pudding ad...,Gunakan spatula untuk mengambil Brighter Skin ...,"Aqua, Glycerin, Tranexamic acid, Butylene glyc...",Semua jenis kulit,https://storage.googleapis.com/skinnie-bucket/...
4,1001,Brighter Sun Day SPF 50 PA++,Joylab,Sun Protection,4.0,73 users,79%,160000,Face companion perfect for a day in the sun!,-,"Water, Ethylhexyl Methoxycinnamate, Niacinamid...",Semua jenis kulit,https://storage.googleapis.com/skinnie-bucket/...


organize data into separate lists

In [22]:
descriptions = [product['product_name'] + ' ' + product['ingredients'] for product in products]
skin_types = [product['suitable_for'] for product in products]
ratings = [product['rate'] for product in products]

vectorization and model

In [23]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(descriptions)
model = LinearRegression()
model.fit(X, ratings)

LinearRegression()

In [24]:
# Save the trained model
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Save the vectorizer
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [25]:
def content_recommendations(user_skin_type):

    with open('model.pkl', 'rb') as f:
        model = pickle.load(f)

    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    user_input = vectorizer.transform([user_skin_type])
    model.predict(user_input)

    skin_types_series = pd.Series(skin_types)

    if user_skin_type == 'Normal':
        filtered_indices = np.where(skin_types_series.isin([user_skin_type, 'Semua jenis kulit']))[0]
    elif user_skin_type == 'Kering':
        filtered_indices = np.where(skin_types_series.isin([user_skin_type, 'Semua jenis kulit']))[0]
    elif user_skin_type == 'Berminyak':
        filtered_indices = np.where(skin_types_series.isin([user_skin_type, 'Semua jenis kulit']))[0]
    else:
        filtered_indices = []

    filtered_products = [products[i] for i in filtered_indices]

    # Convert filtered_products to a DataFrame
    df_filtered_products = pd.DataFrame(filtered_products)

    # Create a column to store the priority for sorting
    df_filtered_products['priority'] = df_filtered_products['suitable_for'].apply(lambda x: 0 if x == user_skin_type else 1)

    # Sort the DataFrame by 'priority', 'suitable_for', and 'rate' columns
    sorted_products = df_filtered_products.sort_values(by=['priority', 'suitable_for', 'rate'], ascending=[True, False, False])

    print("Recommended Products:")
    print("----------------------")
    
    return sorted_products[['product_name', 'suitable_for', 'ingredients', 'rate','brand']]

In [26]:
content_recommendations('Kering')

Recommended Products:
----------------------


,product_name,suitable_for,ingredients,rate,brand
289,Comfort You Green Superfood Serum,Kering,"Niacinamide 5%, Allantoin, Cactus extract, Bet...",5.0,Dermies
1064,Glowing Bright Mild Cleanser Foam,Kering,"Aqua, Disodium Laureth Sulfosuccinate, Cocamid...",5.0,Allura
1320,Hydralife Mild Cleanser Foam,Kering,"Aqua, Disodium Laureth Sulfosuccinate, Cocamid...",5.0,Allura
1824,Milk Mask Organic,Kering,"Bubuk Susu, Gandum, Beras Organik, Madu",5.0,Alca Active Care
1972,Naicinamide & Zinc Serum,Kering,"Niacinamide, Zinc",5.0,MEDGLOW
...,...,...,...,...,...
4077,Brightening Serum,Semua jenis kulit,-,0.0,Moriganic
4080,Brightening Serum,Semua jenis kulit,Aqua DM; Ethyl ascorbic acid; Niacinamide; Pro...,0.0,SRC Dermatiks
4089,Brightening Sunscreen,Semua jenis kulit,-,0.0,SRC Dermatiks
4096,Brightening Toner,Semua jenis kulit,Aqua DM; Butylene glycol; Ethanol; Glycerin; N...,0.0,SRC Dermatiks
